In [ ]:
# Check device number
!ls -ltrh /dev/video*

In [2]:
# # --------------
# # Camera test
# # --------------

# import cv2, time
# from IPython.display import display, clear_output
# from PIL import Image

# cap = cv2.VideoCapture(0, cv2.CAP_V4L2)  # /dev/video0
# cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
# cap.set(cv2.CAP_PROP_FPS, 30)

# for _ in range(100):  # 100프레임만 테스트
#     ok, frame = cap.read()
#     if not ok: 
#         print("프레임 읽기 실패"); break
#     rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#     clear_output(wait=True)
#     display(Image.fromarray(rgb))

# cap.release()


In [ ]:
from flask import Flask, Response
from ultralytics import YOLO
import cv2

# YOLO 모델 불러오기 (best.pt or .engine 가능)
model = YOLO("best.pt")

c_index = 1 #Camera index can be changed

cap = cv2.VideoCapture(c_index) 
assert cap.isOpened(), "Camera open failed."

app = Flask(__name__)

def gen_frames():
    while True:
        ret, frame = cap.read() 
        if not ret:
            break

        # YOLO inference
        results = model.predict(source=frame, imgsz=640, verbose=False)
        annotated = results[0].plot()  # draw inference results

        # JPEG encoding
        ret, buffer = cv2.imencode('.jpg', annotated)
        if not ret:
            continue #If encoding is failed, skip

        # MJPEG streaming: MJPEG = Motion JPEG
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + buffer.tobytes() + b'\r\n')

@app.route('/')
def video_feed():
    return Response(gen_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == "__main__":
    try:
        app.run(host="0.0.0.0", port=8000, debug=False, use_reloader=False)
    finally:
        cap.release()
        cv2.destroyAllWindows()
